In [1]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# 파일 경로 설정
file_path = 'D:/시설/유지/24년11월유지가입자_날짜형식조정.cs'

# 데이터 읽기
df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

# 헤더 값 확인
headers = df.columns.tolist()

# 체크박스로 헤더 선택
header_checkboxes = [widgets.Checkbox(value=False, description=header) for header in headers]
header_box = widgets.VBox(header_checkboxes)
header_label = widgets.Label("삭제할 열을 선택하세요:")
display(header_label, header_box)

# 버튼 생성
delete_columns_button = widgets.Button(description="선택한 열 삭제")
display(delete_columns_button)

# 선택한 행값을 기반으로 삭제
delete_rows_label = widgets.Label("특정 텍스트로 행 삭제:")
row_text_input = widgets.Text(placeholder="행값에 포함된 텍스트를 입력하세요")
delete_rows_button = widgets.Button(description="텍스트 포함된 행 삭제")
display(delete_rows_label, row_text_input, delete_rows_button)

# 최종 저장 버튼
save_button = widgets.Button(description="결과 저장")
save_path_input = widgets.Text(value="D:/시설/유지/필터링_데이터.csv", description="저장 경로:")
display(save_path_input, save_button)

# 출력 영역
output = widgets.Output()
display(output)

# 열 삭제 버튼 클릭 이벤트
def delete_columns_event(b):
    with output:
        clear_output()
        selected_columns = [cb.description for cb in header_checkboxes if cb.value]
        if selected_columns:
            global df
            df = df.drop(columns=selected_columns, errors='ignore')
            print(f"선택한 열이 삭제되었습니다: {selected_columns}")
        else:
            print("선택한 열이 없습니다.")

delete_columns_button.on_click(delete_columns_event)

# 행 삭제 버튼 클릭 이벤트
def delete_rows_event(b):
    with output:
        clear_output()
        text_to_exclude = row_text_input.value
        if text_to_exclude:
            global df
            # 모든 열에서 텍스트 포함된 행 삭제
            df = df[~df.apply(lambda row: row.astype(str).str.contains(text_to_exclude).any(), axis=1)]
            print(f"'{text_to_exclude}'를 포함하는 행이 삭제되었습니다.")
        else:
            print("삭제할 텍스트를 입력하세요.")

delete_rows_button.on_click(delete_rows_event)

# 저장 버튼 클릭 이벤트
def save_data_event(b):
    with output:
        clear_output()
        save_path = save_path_input.value
        if save_path:
            df.to_csv(save_path, index=False, encoding='cp949')
            print(f"데이터가 저장되었습니다: {save_path}")
        else:
            print("저장 경로를 입력하세요.")

save_button.on_click(save_data_event)


Label(value='삭제할 열을 선택하세요:')

Button(description='선택한 열 삭제', style=ButtonStyle())

Label(value='특정 텍스트로 행 삭제:')

Text(value='', placeholder='행값에 포함된 텍스트를 입력하세요')

Button(description='텍스트 포함된 행 삭제', style=ButtonStyle())

Text(value='D:/시설/유지/필터링_데이터.csv', description='저장 경로:')

Button(description='결과 저장', style=ButtonStyle())

Output()

In [5]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# 파일 경로 설정
file_path = 'D:/시설/유지/24년11월유지가입자_날짜형식조정.cs'

# 데이터 읽기
df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

# 헤더 값(열 이름)을 데이터프레임으로 표시
headers = pd.DataFrame({'열 이름': df.columns})
headers.index += 1  # 보기 좋게 1부터 시작하는 인덱스 설정

# 헤더를 테이블로 표시
header_label = widgets.Label("데이터의 헤더(열 이름):")
header_table = widgets.Output()

with header_table:
    clear_output()
    display(headers)

display(header_label, header_table)

# 체크박스로 필요한 열 선택
header_checkboxes = [widgets.Checkbox(value=False, description=header) for header in df.columns]
header_box = widgets.VBox(header_checkboxes)
header_selection_label = widgets.Label("저장할 열을 선택하세요:")
display(header_selection_label, header_box)

# 텍스트 조건 입력
condition_label = widgets.Label("조건에 해당하는 값을 입력하세요 (열 기준으로 검색):")
condition_input = widgets.Text(placeholder="조건 값을 입력하세요")
display(condition_label, condition_input)

# 저장 경로 입력
save_path_input = widgets.Text(value="D:/시설/유지/필터링_결과.csv", description="저장 경로:")
display(save_path_input)

# 저장 버튼 생성
save_button = widgets.Button(description="선택한 열과 조건에 맞는 데이터 저장")
display(save_button)

# 출력 영역
output = widgets.Output()
display(output)

# 날짜 형식 변환 대상 열
date_columns = [
    '계약시작일', '계약종료일', '준공완료일', '관제개통일', '서비스개시일', '서비스재개시일',
    '정지시작일자', '결합시작일', '계약최초서비스게시일', '실적최초등록일'
]

# 저장 버튼 클릭 이벤트
def save_filtered_data(b):
    with output:
        clear_output()
        # 선택한 열
        selected_columns = [cb.description for cb in header_checkboxes if cb.value]
        if not selected_columns:
            print("저장할 열을 선택하세요.")
            return

        # 조건값 입력
        condition_value = condition_input.value.strip()
        if condition_value:
            filtered_df = df[selected_columns]
            # 조건에 해당하는 행 필터링
            filtered_df = filtered_df[filtered_df.apply(
                lambda row: row.astype(str).str.contains(condition_value, na=False).any(), axis=1
            )]
        else:
            filtered_df = df[selected_columns]

        # 날짜 열 변환
        for col in date_columns:
            if col in filtered_df.columns:
                filtered_df[col] = pd.to_datetime(
                    filtered_df[col], format='%Y%m%d', errors='coerce'
                ).dt.strftime('%Y-%m-%d')

        # 저장
        save_path = save_path_input.value.strip()
        if save_path:
            filtered_df.to_csv(save_path, index=False, encoding='cp949')
            print(f"선택한 열과 조건에 맞는 데이터가 저장되었습니다: {save_path}")
        else:
            print("저장 경로를 입력하세요.")

save_button.on_click(save_filtered_data)

Label(value='데이터의 헤더(열 이름):')

Output()

Label(value='저장할 열을 선택하세요:')

Label(value='조건에 해당하는 값을 입력하세요 (열 기준으로 검색):')

Text(value='', placeholder='조건 값을 입력하세요')

Text(value='D:/시설/유지/필터링_결과.csv', description='저장 경로:')

Button(description='선택한 열과 조건에 맞는 데이터 저장', style=ButtonStyle())

Output()

In [7]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox

class DataFilterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("데이터 필터링 및 저장")
       
        # 초기 데이터프레임
        self.df = None
        self.filtered_df = None
       
        # 파일 경로 입력 및 선택
        self.file_label = tk.Label(root, text="파일 경로:")
        self.file_label.grid(row=0, column=0, padx=10, pady=10, sticky='w')
       
        self.file_entry = tk.Entry(root, width=50)
        self.file_entry.grid(row=0, column=1, padx=10, pady=10)
       
        self.browse_button = tk.Button(root, text="파일 선택", command=self.load_file)
        self.browse_button.grid(row=0, column=2, padx=10, pady=10)
       
        # 열 선택 리스트박스
        self.column_label = tk.Label(root, text="열 선택:")
        self.column_label.grid(row=1, column=0, padx=10, pady=10, sticky='nw')
       
        self.column_listbox = tk.Listbox(root, selectmode=tk.MULTIPLE, width=50, height=15)
        self.column_listbox.grid(row=1, column=1, padx=10, pady=10)
       
        # 조건 입력
        self.condition_label = tk.Label(root, text="조건 입력 (포함된 값):")
        self.condition_label.grid(row=2, column=0, padx=10, pady=10, sticky='w')
       
        self.condition_entry = tk.Entry(root, width=50)
        self.condition_entry.grid(row=2, column=1, padx=10, pady=10)
       
        # 저장 경로 입력
        self.save_label = tk.Label(root, text="저장 경로:")
        self.save_label.grid(row=3, column=0, padx=10, pady=10, sticky='w')
       
        self.save_entry = tk.Entry(root, width=50)
        self.save_entry.grid(row=3, column=1, padx=10, pady=10)
       
        self.save_button = tk.Button(root, text="저장 위치 선택", command=self.select_save_path)
        self.save_button.grid(row=3, column=2, padx=10, pady=10)
       
        # 실행 버튼
        self.run_button = tk.Button(root, text="필터링 및 저장", command=self.filter_and_save)
        self.run_button.grid(row=4, column=1, padx=10, pady=10)
   
    def load_file(self):
        # 파일 선택 대화상자 열기
        file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        if file_path:
            self.file_entry.delete(0, tk.END)
            self.file_entry.insert(0, file_path)
            try:
                # 파일 읽기
                self.df = pd.read_csv(file_path, encoding='cp949', low_memory=False)
                self.update_columns()
                messagebox.showinfo("파일 로드 완료", "파일이 성공적으로 로드되었습니다.")
            except Exception as e:
                messagebox.showerror("파일 로드 실패", f"파일을 로드하는 중 오류가 발생했습니다:\n{e}")
   
    def update_columns(self):
        # 열 리스트박스 업데이트
        self.column_listbox.delete(0, tk.END)
        for col in self.df.columns:
            self.column_listbox.insert(tk.END, col)
   
    def select_save_path(self):
        # 저장 경로 선택
        save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
        if save_path:
            self.save_entry.delete(0, tk.END)
            self.save_entry.insert(0, save_path)
   
    def filter_and_save(self):
        if self.df is None:
            messagebox.showerror("데이터 없음", "먼저 파일을 로드하세요.")
            return
       
        # 선택한 열 가져오기
        selected_indices = self.column_listbox.curselection()
        selected_columns = [self.column_listbox.get(i) for i in selected_indices]
       
        if not selected_columns:
            messagebox.showerror("열 선택 없음", "저장할 열을 선택하세요.")
            return
       
        # 조건 가져오기
        condition = self.condition_entry.get().strip()
       
        # 데이터 필터링
        self.filtered_df = self.df[selected_columns]
        if condition:
            self.filtered_df = self.filtered_df[self.filtered_df.apply(
                lambda row: row.astype(str).str.contains(condition, na=False).any(), axis=1
            )]
       
        # 날짜 형식 변환
        date_columns = [
            '계약시작일', '계약종료일', '준공완료일', '관제개통일', '서비스개시일', '서비스재개시일',
            '정지시작일자', '결합시작일', '계약최초서비스게시일', '실적최초등록일'
        ]
        for col in date_columns:
            if col in self.filtered_df.columns:
                self.filtered_df[col] = pd.to_datetime(
                    self.filtered_df[col], format='%Y%m%d', errors='coerce'
                ).dt.strftime('%Y-%m-%d')
       
        # 저장 경로
        save_path = self.save_entry.get().strip()
        if not save_path:
            messagebox.showerror("저장 경로 없음", "저장 경로를 입력하세요.")
            return
       
        try:
            # 데이터 저장
            self.filtered_df.to_csv(save_path, index=False, encoding='cp949')
            messagebox.showinfo("저장 완료", f"데이터가 성공적으로 저장되었습니다:\n{save_path}")
        except Exception as e:
            messagebox.showerror("저장 실패", f"데이터를 저장하는 중 오류가 발생했습니다:\n{e}")

# GUI 실행
root = tk.Tk()
app = DataFilterApp(root)
root.mainloop()

In [9]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox

class DataFilterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("데이터 필터링 및 분석")
       
        # 초기 데이터프레임
        self.df = None
        self.filtered_df = None
       
        # 파일 경로 입력 및 선택
        self.file_label = tk.Label(root, text="파일 경로:")
        self.file_label.grid(row=0, column=0, padx=10, pady=10, sticky='w')
       
        self.file_entry = tk.Entry(root, width=50)
        self.file_entry.grid(row=0, column=1, padx=10, pady=10)
       
        self.browse_button = tk.Button(root, text="파일 선택", command=self.load_file)
        self.browse_button.grid(row=0, column=2, padx=10, pady=10)
       
        # 열 선택 리스트박스
        self.column_label = tk.Label(root, text="열 선택:")
        self.column_label.grid(row=1, column=0, padx=10, pady=10, sticky='nw')
       
        self.column_listbox = tk.Listbox(root, selectmode=tk.SINGLE, width=50, height=10)
        self.column_listbox.grid(row=1, column=1, padx=10, pady=10)
       
        self.apply_column_button = tk.Button(root, text="열 적용", command=self.apply_column_selection)
        self.apply_column_button.grid(row=1, column=2, padx=10, pady=10)
       
        # 특정 텍스트 조건 입력
        self.condition_label = tk.Label(root, text="특정 텍스트 값:")
        self.condition_label.grid(row=2, column=0, padx=10, pady=10, sticky='w')
       
        self.condition_entry = tk.Entry(root, width=50)
        self.condition_entry.grid(row=2, column=1, padx=10, pady=10)
       
        # 저장 경로 입력
        self.save_label = tk.Label(root, text="저장 경로:")
        self.save_label.grid(row=3, column=0, padx=10, pady=10, sticky='w')
       
        self.save_entry = tk.Entry(root, width=50)
        self.save_entry.grid(row=3, column=1, padx=10, pady=10)
       
        self.save_button = tk.Button(root, text="저장 위치 선택", command=self.select_save_path)
        self.save_button.grid(row=3, column=2, padx=10, pady=10)
       
        # 실행 버튼
        self.run_button = tk.Button(root, text="필터링 및 저장", command=self.filter_and_save)
        self.run_button.grid(row=4, column=1, padx=10, pady=10)
       
        # 출력 영역
        self.output_label = tk.Label(root, text="")
        self.output_label.grid(row=5, column=0, columnspan=3, padx=10, pady=10)
   
    def load_file(self):
        # 파일 선택 대화상자 열기
        file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        if file_path:
            self.file_entry.delete(0, tk.END)
            self.file_entry.insert(0, file_path)
            try:
                # 파일 읽기
                self.df = pd.read_csv(file_path, encoding='cp949', low_memory=False)
                self.update_columns()
                messagebox.showinfo("파일 로드 완료", "파일이 성공적으로 로드되었습니다.")
            except Exception as e:
                messagebox.showerror("파일 로드 실패", f"파일을 로드하는 중 오류가 발생했습니다:\n{e}")
   
    def update_columns(self):
        # 열 리스트박스 업데이트
        self.column_listbox.delete(0, tk.END)
        for col in self.df.columns:
            self.column_listbox.insert(tk.END, col)
   
    def apply_column_selection(self):
        if self.df is None:
            messagebox.showerror("데이터 없음", "먼저 파일을 로드하세요.")
            return
       
        # 선택한 열 가져오기
        selected_index = self.column_listbox.curselection()
        if not selected_index:
            messagebox.showerror("열 선택 없음", "적용할 열을 선택하세요.")
            return
       
        selected_column = self.column_listbox.get(selected_index)
       
        # 열에서 500개 이상 중복된 값 필터링
        filtered_values = self.df[selected_column].value_counts()
        filtered_values = filtered_values[filtered_values >= 500].index.tolist()
       
        if not filtered_values:
            self.output_label.config(text="500개 이상 중복된 값이 없습니다.")
        else:
            self.filtered_df = self.df[self.df[selected_column].isin(filtered_values)]
            self.output_label.config(
                text=f"'{selected_column}' 열에서 500개 이상 중복된 값이 필터링되었습니다."
            )
   
    def select_save_path(self):
        # 저장 경로 선택
        save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
        if save_path:
            self.save_entry.delete(0, tk.END)
            self.save_entry.insert(0, save_path)
   
    def filter_and_save(self):
        if self.filtered_df is None:
            messagebox.showerror("필터링 안됨", "먼저 열을 적용하고 데이터를 필터링하세요.")
            return
       
        # 특정 텍스트 조건 추가
        condition = self.condition_entry.get().strip()
        if condition:
            self.filtered_df = self.filtered_df[
                self.filtered_df.apply(lambda row: row.astype(str).str.contains(condition, na=False).any(), axis=1)
            ]
       
        # 저장 경로
        save_path = self.save_entry.get().strip()
        if not save_path:
            messagebox.showerror("저장 경로 없음", "저장 경로를 입력하세요.")
            return
       
        try:
            # 데이터 저장
            self.filtered_df.to_csv(save_path, index=False, encoding='cp949')
            messagebox.showinfo("저장 완료", f"데이터가 성공적으로 저장되었습니다:\n{save_path}")
        except Exception as e:
            messagebox.showerror("저장 실패", f"데이터를 저장하는 중 오류가 발생했습니다:\n{e}")

# GUI 실행
root = tk.Tk()
app = DataFilterApp(root)
root.mainloop()

In [11]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# 파일 경로 설정
file_path = 'D:/시설/유지/24년11월유지가입자_날짜형식조정.cs'

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df = pd.read_csv(file_path, encoding='cp949', dtype=str)
df.fillna('', inplace=True)  # NaN 값을 빈 문자열로 대체

# 입력 위젯 설정
input_fields = {
    '청구번호': '청구 번호 입력',
    '계약번호': '계약 번호 입력',
    '상호': '상호 부분 입력',
    '실적최초등록일': '실적최초등록일 부분 입력',
    '서비스재개시일': '서빗재개시일 부분 입력',
    '고객명': '고객명 부분 입력',
    '설치주소': '설치 주소 부분 입력',
    '관리본부명': '관리 본부명 입력',
    '관리지사명': '관리 지사명 입력',
    '시설구분': '시설대상 입력',
    '요금구분': '요금대상 입력',
    '제외사유': '제외사유 대상입력',
    '영업자명': '구역 담당 영업사원 입력',
    '무인매장구분': '무인매장구분 입력',
    '무인매장업종구분': '무인매장업종구분 입력'
}

# 제외할 컬럼 목록 정의
excluded_columns = [
    "관리본부코드", "관리지사코드", "영상전환일", "서비스상태(대)", "서비스상태(중)",
    "접속전화번호", "설치우편번호", "견적월정료", "할인율", "월정료면제사유",
    "면제시작일", "면제종료일", "계약보증금", "보증금면제사유", "판매상품비", "견적설치공사비",
    "설치공사할부개월", "계약공사비", "면제공사비", "실징수액", "추천구분", "영업본부코드",
    "영업지사코드", "영업지점명", "모집유형", "추천본부코드", "추천본부명", "추천지사코드",
    "유통망소분류", "추천채널", "정보제공자부서", "정보제공자사번", "정보제공자연락처",
    "청구본부코드", "청구지사코드", "청구자명", "세금계산서발행여부", "계산서발행여부", "선후납구분",
    "청구전화번호", "청구우편번호", "청약일자", "청약취소일자", "공사희망일", "공사완료일",
    "준공완료일", "관제개통일", "서비스개시일", "공사의뢰유형", "회선방식(대)",
    "MUX", "전용회선번호", "MIN번호", "해지일자", "업종_대", "업종_중",
    "비고", "결합구분", "결합상품명", "결합약정개월수", "결합시작일", "금융기관코드",
    "면책구분", "타사전환", "이전고객번호", "전입신규여부", "고객추가정보", "청약취소사유",
    "정지기간", "정지유형", "설치공사비", "영업이익율", "공헌이익율","SW주장치안정화DB",
    "선장품", "브랜드", "A/S운용기간", "고객계약유형","판매할부개월",
    "최종계약구분", "KT마스터ID", "BIZNARU계약ID", "정지희망종료일", "신규PP","직접/추천",
    "판매구분"
]

# 데이터프레임에서 이 컬럼들을 제거
df = df.drop(columns=[col for col in excluded_columns if col in df.columns])

inputs = {key: widgets.Text(placeholder=desc, description=f'{key}:') for key, desc in input_fields.items()}

# 체크박스와 컬럼 이름 구성
checkboxes = [widgets.Checkbox(value=False, description=label) for label in df.columns]
checkbox_container = widgets.VBox(children=[widgets.HBox(checkboxes[i:i+5]) for i in range(0, len(checkboxes), 5)])

# 전체 선택/해제 버튼
toggle_buttons = widgets.ToggleButtons(
    options=['Select All', 'Deselect All'],
    description='Check:',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
)

def toggle_checkboxes(change):
    if change['new'] == 'Select All':
        for checkbox in checkboxes:
            checkbox.value = True
    elif change['new'] == 'Deselect All':
        for checkbox in checkboxes:
            checkbox.value = False
    toggle_buttons.value = None  # Reset toggle button state

toggle_buttons.observe(toggle_checkboxes, 'value')

# 실행 버튼
run_button = widgets.Button(description='조회 실행', button_style='success', icon='search')
# 저장 버튼
save_button = widgets.Button(description='Excel로 저장', button_style='info', icon='save')

output = widgets.Output()

def run_query(b):
    global filtered_data, selected_columns
    with output:
        clear_output(wait=True)
        try:
            condition = True
            for field, widget in inputs.items():
                if widget.value.strip():
                    condition &= df[field].str.contains(widget.value.strip(), case=False)
            filtered_data = df.loc[condition]
            selected_columns = [cb.description for cb in checkboxes if cb.value]
            if not selected_columns:
                raise ValueError("결과로 표시할 컬럼을 선택하세요.")
            if filtered_data.empty:
                raise ValueError("조회된 데이터가 없습니다. 조건을 확인해주세요.")
            styled_html = filtered_data[selected_columns].to_html(escape=False, classes='styledTable')
            styled_css = """
            <style>
                .styledTable {
                    width: 100%;
                    border-collapse: collapse;
                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                    background-color: #f9f9f9;
                    color: #333;
                }
                .styledTable th, .styledTable td {
                    border: 1px solid #dddddd;
                    padding: 8px;
                    text-align: left;
                }
                .styledTable th {
                    background-color: #f2f2f2;
                    color: #333;
                }
                .styledTable tr:nth-child(even) {
                    background-color: #ffffff;
                }
                .styledTable tr:nth-child(odd) {
                    background-color: #f7f7f7;
                }
                .styledTable tr:hover {
                    background-color: #f1f1f1;
                }
            </style>
            """
            display(HTML(styled_css + styled_html))
        except Exception as e:
            display(HTML(f"<div style='color: red;'>Error: {e}</div>"))

def save_to_excel(b):
    global filtered_data, selected_columns
    try:
        if 'filtered_data' in globals() and 'selected_columns' in globals():
            filtered_data[selected_columns].to_excel('추출리스트.xlsx', index=False)
            with output:
                clear_output(wait=True)
                display(HTML("<div style='color: green;'>Data successfully saved to '추출리스트.xlsx'</div>"))
        else:
            raise ValueError("저장할 데이터가 없습니다. 먼저 조회를 실행하세요.")
    except Exception as e:
        with output:
            clear_output(wait=True)
            display(HTML(f"<div style='color: red;'>Error saving file: {e}</div>"))

# 이벤트 핸들러 연결
run_button.on_click(run_query)
save_button.on_click(save_to_excel)

# 입력 필드와 버튼을 같은 행에 배치
input_row = widgets.HBox([widgets.VBox([inputs[field] for field in inputs.keys()]), widgets.VBox([run_button, save_button])])

# 체크박스 토글과 컬럼 선택을 관리하는 행
checkbox_toggle_row = widgets.HBox([toggle_buttons])

# UI 구성
ui = widgets.VBox([input_row, checkbox_toggle_row, checkbox_container, output])
display(ui)